In [1]:
import numpy as np
import pandas as pd
import os 
import time
import deepdish.io as dd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

from svi import SVI
import models as m
import datasets as d

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
data = dd.load("sequenced_data_for_VAE_lenght-160_stride-10.h5")

In [4]:
X,y = data["X"], data["y"]

In [5]:
X

tensor([[0.1872, 0.1890, 0.1805,  ..., 0.1791, 0.1756, 0.1754],
        [0.1980, 0.1989, 0.1974,  ..., 0.1850, 0.1930, 0.1860],
        [0.2103, 0.2107, 0.2080,  ..., 0.1960, 0.2044, 0.2041],
        ...,
        [0.4694, 0.4644, 0.4707,  ..., 0.4877, 0.4774, 0.4771],
        [0.5260, 0.5163, 0.5141,  ..., 0.4935, 0.5005, 0.5110],
        [0.5704, 0.5643, 0.5629,  ..., 0.5024, 0.4991, 0.5269]],
       dtype=torch.float64)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=666, shuffle=True)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
X_train = torch.tensor(X_train.astype('float32'))
X_test = torch.tensor(X_test.astype('float32'))

In [9]:
X_train = X_train.to(device)
X_test = X_test.to(device)

# standartní VAE

In [21]:
VAE = torch.load("VAE-saved_model_and_losses/DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4.pt")

In [22]:
VAE.eval()

DeepDenseVAE_mark_V(
  (encoder): Sequential(
    (0): Linear(in_features=160, out_features=256, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ELU(alpha=1.0)
    (4): VariationalLayer(
      (mu): Linear(in_features=128, out_features=15, bias=True)
      (rho): Linear(in_features=128, out_features=15, bias=True)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=15, out_features=128, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ELU(alpha=1.0)
    (4): VariationalDecoderOutput(
      (mu): Linear(in_features=256, out_features=160, bias=True)
      (rho): Linear(in_features=256, out_features=1, bias=True)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
)

In [35]:
Z_hat, mu_tr, sigma_tr = VAE.encoder(X_train.to(device))
mu_train = mu_tr.cpu().detach().numpy()

Z_hat, mu_te, sigma_te = VAE.encoder(X_test.to(device))
mu_test = mu_te.cpu().detach().numpy()

In [42]:
clf = SVC(kernel="rbf", C=100, gamma=0.1, verbose=True)

In [43]:
clf.fit(mu_train, y_train)

[LibSVM]

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [44]:
y_pred = clf.predict(mu_test)

In [46]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[2767, 1329,   26,  113],
       [ 543, 9704,   46,  152],
       [  35,   67,  355,   22],
       [ 248,  210,   13, 1106]], dtype=int64)

In [49]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8324569789674953

In [1]:
dd.save("VAE-saved_model_and_losses/SVM-rbf_on_VAE_latent_space.h5", clf)

NameError: name 'dd' is not defined

In [50]:
clf_linear = SVC(kernel="linear", C=100, verbose=True)

In [ ]:
clf_linear.fit(mu_train, y_train)

[LibSVM]

# CNN-VAE (elu)

In [10]:
CNNVAE1 = torch.load("VAE-saved_model_and_losses/DeepConvVAE_ELU_NLL_250ep_lr-{1e-3, 1e-4}_bs-300.pt")

In [11]:
CNNVAE1.eval()

DeepConvVAE_ELU(
  (encoder): Sequential(
    (0): Reshape()
    (1): Conv1d(1, 16, kernel_size=(6,), stride=(2,))
    (2): ELU(alpha=1.0)
    (3): Conv1d(16, 32, kernel_size=(4,), stride=(2,))
    (4): ELU(alpha=1.0)
    (5): Conv1d(32, 64, kernel_size=(4,), stride=(2,))
    (6): ELU(alpha=1.0)
    (7): Flatten()
    (8): VariationalLayer(
      (mu): Linear(in_features=1152, out_features=15, bias=True)
      (rho): Linear(in_features=1152, out_features=15, bias=True)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=15, out_features=1152, bias=True)
    (1): ELU(alpha=1.0)
    (2): Reshape()
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,))
    (4): ELU(alpha=1.0)
    (5): ConvTranspose1d(32, 16, kernel_size=(4,), stride=(2,))
    (6): ELU(alpha=1.0)
    (7): ConvTransposeDecoderOutput(
      (mu): ConvTranspose1d(16, 1, kernel_size=(6,), stride=(2,))
      (rho): Linear(in_features=1248, out_features=1, bias

In [12]:
Z_hat, mu_tr, sigma_tr = CNNVAE1.encoder(X_train)
mu_train = mu_tr.cpu().detach().numpy()

Z_hat, mu_te, sigma_te = CNNVAE1.encoder(X_test)
mu_test = mu_te.cpu().detach().numpy()

In [13]:
CNNVAE1clf_rbf = SVC(kernel="rbf", C=100, gamma=0.1, verbose=True)

In [14]:
CNNVAE1clf_rbf.fit(mu_train, y_train)

[LibSVM]

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [15]:
y_pred = CNNVAE1clf_rbf.predict(mu_test)

In [16]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[2673, 1398,   30,  134],
       [ 383, 9878,   39,  145],
       [  36,   57,  364,   22],
       [ 234,  174,   17, 1152]], dtype=int64)

In [17]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8405234225621415

In [18]:
dd.save("VAE-saved_model_and_losses/SVM-rbf_on_CNNVAE-elu_latent_space.h5", CNNVAE1clf_rbf)

# CNN-VAE (relu)

In [10]:
CNNVAE2 = torch.load("VAE-saved_model_and_losses/DeepConvVAE_ReLU_NLL_200ep_lr-1e-4_bs-100.pt")

In [11]:
CNNVAE2.eval()

DeepConvVAE(
  (encoder): Sequential(
    (0): Reshape()
    (1): Conv1d(1, 16, kernel_size=(6,), stride=(2,))
    (2): ReLU()
    (3): Conv1d(16, 32, kernel_size=(4,), stride=(2,))
    (4): ReLU()
    (5): Conv1d(32, 64, kernel_size=(4,), stride=(2,))
    (6): ReLU()
    (7): Flatten()
    (8): VariationalLayer(
      (mu): Linear(in_features=1152, out_features=15, bias=True)
      (rho): Linear(in_features=1152, out_features=15, bias=True)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=15, out_features=1152, bias=True)
    (1): ReLU()
    (2): Reshape()
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,))
    (4): ReLU()
    (5): ConvTranspose1d(32, 16, kernel_size=(4,), stride=(2,))
    (6): ReLU()
    (7): ConvTransposeDecoderOutput(
      (mu): ConvTranspose1d(16, 1, kernel_size=(6,), stride=(2,))
      (rho): Linear(in_features=1248, out_features=1, bias=True)
      (flatten): Flatten()
      (flatten_mu)

In [12]:
Z_hat, mu_tr, sigma_tr = CNNVAE2.encoder(X_train)
mu_train = mu_tr.cpu().detach().numpy()

Z_hat, mu_te, sigma_te = CNNVAE2.encoder(X_test)
mu_test = mu_te.cpu().detach().numpy()

In [13]:
CNNVAE2clf_rbf = SVC(kernel="rbf", C=100, gamma=0.1, verbose=True)

In [14]:
CNNVAE2clf_rbf.fit(mu_train, y_train)

[LibSVM]

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [15]:
y_pred = CNNVAE2clf_rbf.predict(mu_test)

In [16]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[2651, 1435,   25,  124],
       [ 335, 9933,   38,  139],
       [  41,   53,  367,   18],
       [ 218,  158,   11, 1190]], dtype=int64)

In [17]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8449450286806883

In [18]:
dd.save("VAE-saved_model_and_losses/SVM-rbf_on_CNNVAE-relu_latent_space.h5", CNNVAE2clf_rbf)

# LSTM VAE

In [19]:
mu_1 = np.load("VAE_saves_and_logs/mu_lstm1.npy")
mu_2 = np.load("VAE_saves_and_logs/mu_lstm2.npy")
mu_3 = np.load("VAE_saves_and_logs/mu_lstm3.npy")
mu_test = np.load("VAE_saves_and_logs/mu_lstm_test.npy")

In [21]:
mu_train = np.concatenate((mu_1, mu_2, mu_3))

In [22]:
LSTMVAEclf_rbf = SVC(kernel="rbf", C=100, gamma=0.1, verbose=True)

In [23]:
LSTMVAEclf_rbf.fit(mu_train, y_train)

[LibSVM]

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)

In [24]:
y_pred = LSTMVAEclf_rbf.predict(mu_test)

In [25]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[ 2306,  1785,    26,   118],
       [  298, 10010,    39,    98],
       [   24,    35,   401,    19],
       [  184,   152,    18,  1223]], dtype=int64)

In [26]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8329349904397706

In [27]:
dd.save("VAE-saved_model_and_losses/SVM-rbf_on_LSTM-VAE_latent_space.h5", LSTMVAEclf_rbf)